In [3]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy

In [4]:
def generate_random_graph(num_nodes, probability):
    graph = [[0] * num_nodes for _ in range(num_nodes)]
    
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if random.random() < probability:
                graph[i][j] = 1
                graph[j][i] = 1

    return graph

def generate_random_solution(num_nodes):
    nodes = list(range(num_nodes))
    random.shuffle(nodes)
    return nodes

def calculate_total_edge_length(graph, solution):
    total_length = 0
    for i in range(len(graph)):
        for j in range(i + 1, len(graph)):
            if graph[i][j] == 1:
                position_i = solution.index(i)
                position_j = solution.index(j)
                total_length += abs(position_i - position_j)
    return total_length

# Example usage:
num_nodes = 10
edge_probability = 0.5

# Generate a random graph
graph = generate_random_graph(num_nodes, edge_probability)

# Generate a random solution
random_solution = generate_random_solution(num_nodes)

# Calculate the total edge length for the random solution
total_edge_length = calculate_total_edge_length(graph, random_solution)

print("Random Graph (Adjacency Matrix):")
for row in graph:
    print(row)

print("\nRandom Solution (Permutation):", random_solution)
print("Total Edge Length:", total_edge_length)

Random Graph (Adjacency Matrix):
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 1, 1, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 1, 1, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 0, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 1, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 1, 1, 1, 1, 0, 0]

Random Solution (Permutation): [9, 1, 4, 0, 8, 2, 7, 5, 3, 6]
Total Edge Length: 63


In [5]:
import itertools

def brute_force(graph):
    num_nodes = len(graph)
    best_solution = None
    best_length = float('inf')

    # Generate all permutations of nodes
    all_permutations = itertools.permutations(range(num_nodes))

    # Check total edge length for each permutation
    for solution in all_permutations:
        total_length = calculate_total_edge_length(graph, solution)
        if total_length < best_length:
            best_length = total_length
            best_solution = solution

    return best_solution, best_length

In [6]:
brute_force(graph)

((0, 6, 8, 3, 4, 1, 9, 5, 7, 2), 33)

In [7]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = random.sample(range(len(solution)), 2)
    
    # Swap the positions of the selected nodes
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [8]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    # Reverse the subset of nodes between index1 and index2
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [9]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    # Find the largest index k such that a[k] < a[k+1]
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    # If no such index exists, the permutation is the last one
    if k == -1:
        return sorted(new_solution)

    # Find the largest index l greater than k such that a[k] < a[l]
    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    # Swap a[k] and a[l]
    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    # Reverse the sequence from a[k+1] up to and including the final element a[n-1]
    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [10]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [11]:
def local_search_first_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [12]:
def local_search_best_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [13]:
local_search(graph, random_solution, total_edge_length, 1000, change_func=make_change_swap)

[9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
[9, 1, 4, 5, 8, 2, 7, 0, 3, 6] 55 1
[9, 1, 4, 5, 8, 6, 7, 0, 3, 2] 49 2
[9, 1, 4, 5, 8, 6, 7, 0, 3, 2] 49 3
[4, 1, 9, 5, 8, 6, 7, 0, 3, 2] 47 4
[4, 1, 9, 5, 8, 6, 0, 7, 3, 2] 45 5
[4, 1, 9, 5, 8, 6, 0, 7, 3, 2] 45 6
[5, 1, 9, 4, 8, 6, 0, 7, 3, 2] 39 7
[5, 1, 9, 4, 8, 6, 0, 7, 3, 2] 39 8
[5, 1, 9, 4, 8, 6, 0, 7, 3, 2] 39 9
[5, 1, 9, 4, 8, 6, 0, 7, 3, 2] 39 10
[5, 1, 9, 4, 8, 6, 0, 7, 3, 2] 39 11
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 12
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 13
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 14
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 15
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 16
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 17
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 18
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 19
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 20
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 21
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 22
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 23
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 24
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 25
[5, 1, 9, 4, 3, 6, 0, 7, 8, 2] 37 26
[5, 1, 9, 4

([5, 1, 9, 4, 3, 7, 0, 6, 8, 2], 35, 106)

In [14]:
local_search(graph, random_solution, total_edge_length, 1000, change_func=make_change_inverse)

[9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
[9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 1
[9, 5, 7, 2, 8, 0, 4, 1, 3, 6] 55 2
[9, 5, 7, 2, 8, 0, 4, 1, 3, 6] 55 3
[9, 5, 7, 2, 8, 0, 4, 1, 3, 6] 55 4
[9, 5, 7, 2, 8, 0, 4, 1, 3, 6] 55 5
[9, 5, 7, 2, 8, 0, 4, 1, 3, 6] 55 6
[9, 5, 6, 3, 1, 4, 0, 8, 2, 7] 48 7
[9, 5, 6, 3, 1, 4, 0, 8, 2, 7] 48 8
[9, 5, 6, 3, 1, 4, 0, 8, 2, 7] 48 9
[9, 5, 6, 3, 1, 4, 0, 8, 2, 7] 48 10
[9, 5, 6, 3, 1, 4, 0, 8, 2, 7] 48 11
[9, 5, 6, 3, 1, 4, 0, 8, 2, 7] 48 12
[2, 8, 0, 4, 1, 3, 6, 5, 9, 7] 38 13
[2, 8, 0, 4, 1, 3, 6, 5, 9, 7] 38 14
[2, 8, 0, 4, 1, 3, 6, 5, 9, 7] 38 15
[2, 8, 0, 4, 1, 3, 6, 5, 9, 7] 38 16
[2, 8, 0, 4, 1, 3, 6, 5, 9, 7] 38 17
[2, 8, 0, 4, 1, 3, 6, 5, 9, 7] 38 18
[2, 8, 0, 4, 1, 3, 6, 5, 9, 7] 38 19
[2, 8, 0, 4, 1, 3, 6, 5, 9, 7] 38 20
[2, 8, 0, 4, 6, 3, 1, 5, 9, 7] 36 21
[2, 8, 0, 4, 6, 3, 1, 5, 9, 7] 36 22
[2, 8, 0, 4, 6, 3, 1, 5, 9, 7] 36 23
[2, 8, 0, 4, 6, 3, 1, 5, 9, 7] 36 24
[2, 8, 0, 4, 6, 3, 1, 5, 9, 7] 36 25
[2, 8, 0, 4, 6, 3, 1, 5, 9, 7] 36 26
[2, 8, 0, 4

([2, 8, 0, 6, 4, 3, 1, 9, 5, 7], 33, 68)

In [15]:
def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i



In [16]:
local_search_with_permutation(graph, random_solution, total_edge_length, 1000, change_func=make_change_inverse)

[9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
[9, 3, 5, 7, 2, 8, 1, 4, 0, 6] 62 1
[9, 3, 5, 7, 2, 4, 1, 8, 0, 6] 56 2
[3, 9, 5, 7, 2, 4, 1, 8, 0, 6] 55 3
[3, 9, 5, 1, 4, 2, 7, 8, 0, 6] 54 4
[3, 9, 5, 1, 4, 0, 8, 7, 6, 2] 52 5
[3, 9, 5, 1, 4, 6, 7, 8, 0, 2] 48 6
[3, 9, 7, 6, 4, 1, 5, 8, 0, 2] 46 7
[3, 9, 7, 4, 6, 8, 1, 0, 5, 2] 45 8
[3, 9, 7, 4, 6, 8, 1, 0, 5, 2] 45 9
[3, 9, 7, 4, 6, 8, 1, 0, 5, 2] 45 10
[3, 9, 7, 4, 6, 8, 1, 0, 5, 2] 45 11
[3, 9, 7, 4, 6, 8, 1, 0, 5, 2] 45 12
[3, 9, 7, 4, 6, 8, 1, 0, 5, 2] 45 13
[3, 9, 7, 4, 6, 8, 1, 0, 5, 2] 45 14
[3, 6, 4, 7, 9, 8, 1, 0, 5, 2] 44 15
[6, 3, 4, 8, 0, 1, 5, 9, 7, 2] 42 16
[6, 3, 4, 8, 0, 1, 5, 9, 7, 2] 42 17
[6, 3, 4, 8, 0, 1, 5, 9, 7, 2] 42 18
[6, 3, 4, 8, 0, 1, 5, 9, 7, 2] 42 19
[6, 3, 4, 8, 0, 1, 5, 9, 7, 2] 42 20
[6, 3, 4, 8, 0, 1, 9, 5, 7, 2] 41 21
[2, 7, 5, 9, 1, 3, 0, 4, 6, 8] 35 22
[2, 7, 9, 5, 1, 3, 4, 0, 6, 8] 34 23
[2, 7, 9, 5, 1, 3, 4, 0, 6, 8] 34 24
[2, 7, 9, 5, 1, 3, 4, 0, 6, 8] 34 25
[2, 7, 9, 5, 1, 3, 4, 0, 6, 8] 34 26
[2, 7, 9, 5

([8, 6, 0, 4, 3, 1, 9, 5, 7, 2], 33, 56)

In [17]:
local_search_with_permutation(graph, random_solution, total_edge_length, 1000, change_func=make_change_swap)

[9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
[2, 1, 4, 0, 8, 9, 7, 5, 3, 6] 52 1
[0, 1, 4, 2, 9, 3, 5, 6, 7, 8] 49 2
[5, 1, 4, 2, 9, 3, 0, 6, 7, 8] 45 3
[5, 1, 4, 2, 9, 3, 7, 0, 6, 8] 43 4
[5, 1, 4, 2, 9, 3, 7, 0, 6, 8] 43 5
[5, 1, 4, 3, 0, 6, 8, 9, 7, 2] 42 6
[5, 1, 4, 8, 0, 6, 9, 3, 7, 2] 41 7
[5, 1, 4, 0, 8, 9, 6, 3, 7, 2] 40 8
[5, 1, 4, 0, 8, 9, 6, 3, 7, 2] 40 9
[5, 1, 4, 0, 8, 9, 6, 3, 7, 2] 40 10
[5, 1, 4, 0, 8, 9, 6, 3, 7, 2] 40 11
[5, 1, 4, 3, 9, 0, 6, 7, 8, 2] 39 12
[5, 1, 4, 3, 9, 0, 6, 7, 8, 2] 39 13
[5, 1, 4, 3, 9, 7, 6, 0, 8, 2] 37 14
[5, 1, 4, 3, 9, 7, 6, 0, 8, 2] 37 15
[5, 1, 4, 3, 9, 7, 6, 0, 8, 2] 37 16
[5, 1, 4, 3, 9, 7, 6, 0, 8, 2] 37 17
[5, 1, 4, 3, 9, 7, 6, 0, 8, 2] 37 18
[5, 1, 4, 3, 9, 7, 6, 0, 8, 2] 37 19
[5, 1, 4, 9, 3, 7, 6, 0, 8, 2] 36 20
[5, 1, 4, 9, 3, 7, 6, 0, 8, 2] 36 21
[5, 1, 4, 9, 3, 7, 6, 0, 8, 2] 36 22
[5, 1, 4, 9, 3, 7, 6, 0, 8, 2] 36 23
[5, 1, 4, 9, 3, 7, 6, 0, 8, 2] 36 24
[5, 1, 4, 9, 3, 7, 6, 0, 8, 2] 36 25
[5, 1, 4, 9, 3, 7, 6, 0, 8, 2] 36 26
[5, 1, 4, 9

([5, 1, 9, 4, 3, 7, 6, 0, 8, 2], 35, 48)

In [18]:
def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        elif random.random() < 1 / (i**0.5):
            print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [19]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)

[9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63
divs
[9, 5, 4, 0, 8, 2, 7, 1, 3, 6] 63
[5, 9, 4, 0, 8, 2, 7, 1, 3, 6] 60
divs
[5, 3, 4, 0, 8, 2, 7, 1, 9, 6] 67
divs
[5, 4, 3, 0, 8, 2, 7, 1, 9, 6] 67
[7, 4, 3, 0, 8, 2, 5, 1, 9, 6] 59
[7, 4, 3, 0, 8, 2, 5, 1, 9, 6] 59
divs
[3, 4, 7, 0, 8, 2, 5, 1, 9, 6] 61
[3, 4, 7, 0, 8, 2, 5, 1, 9, 6] 61
[3, 4, 7, 0, 8, 2, 5, 1, 9, 6] 61
[3, 4, 7, 0, 8, 2, 1, 5, 9, 6] 58
[3, 4, 7, 0, 8, 2, 1, 5, 9, 6] 58
divs
[3, 4, 7, 0, 2, 8, 1, 5, 9, 6] 58
divs
[3, 4, 0, 7, 2, 8, 1, 5, 9, 6] 60
[3, 4, 0, 7, 2, 8, 1, 5, 9, 6] 60
[3, 7, 0, 4, 2, 8, 1, 5, 9, 6] 56
[3, 7, 0, 4, 2, 8, 1, 5, 9, 6] 56
[3, 7, 0, 4, 2, 8, 1, 5, 9, 6] 56
[3, 7, 0, 4, 2, 8, 1, 5, 9, 6] 56
[3, 7, 0, 4, 2, 8, 1, 5, 9, 6] 56
divs
[3, 7, 0, 4, 2, 5, 1, 8, 9, 6] 58
divs
[1, 7, 0, 4, 2, 5, 3, 8, 9, 6] 64
[1, 7, 0, 4, 2, 5, 3, 8, 9, 6] 64
[1, 7, 0, 9, 2, 5, 3, 8, 4, 6] 59
[1, 7, 0, 9, 2, 5, 3, 8, 4, 6] 59
[1, 7, 0, 9, 2, 5, 3, 8, 4, 6] 59
[1, 7, 0, 9, 2, 5, 3, 8, 4, 6] 59
[1, 2, 0, 9, 7, 5, 3, 8, 4, 6] 55
[1, 6, 0

([5, 1, 9, 4, 7, 3, 8, 6, 0, 2], 35, 212)

In [20]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)

[9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63
divs
[9, 0, 4, 1, 8, 2, 7, 5, 3, 6] 63
[0, 9, 4, 1, 8, 2, 7, 5, 3, 6] 60
[0, 9, 4, 1, 8, 5, 7, 2, 3, 6] 56
[0, 9, 4, 1, 8, 5, 7, 2, 3, 6] 56
[0, 9, 4, 1, 8, 5, 7, 2, 3, 6] 56
[0, 6, 3, 2, 7, 5, 8, 1, 4, 9] 52
divs
[0, 6, 3, 8, 5, 7, 2, 1, 4, 9] 54
[0, 6, 3, 8, 7, 5, 2, 1, 4, 9] 52
[0, 6, 3, 8, 7, 5, 2, 1, 4, 9] 52
divs
[0, 6, 3, 4, 1, 2, 5, 7, 8, 9] 53
[0, 6, 3, 4, 1, 2, 8, 7, 5, 9] 49
[0, 6, 3, 4, 8, 2, 1, 7, 5, 9] 47
[0, 6, 8, 4, 3, 2, 1, 7, 5, 9] 43
[0, 6, 8, 4, 3, 2, 1, 7, 5, 9] 43
[0, 6, 8, 4, 3, 2, 1, 7, 5, 9] 43
[0, 6, 8, 4, 3, 2, 1, 7, 5, 9] 43
[0, 6, 8, 4, 3, 2, 1, 7, 5, 9] 43
[0, 6, 8, 4, 3, 2, 1, 7, 5, 9] 43
[0, 6, 8, 4, 3, 2, 1, 7, 5, 9] 43
[0, 6, 8, 4, 3, 2, 1, 7, 5, 9] 43
[0, 6, 8, 4, 3, 2, 1, 7, 5, 9] 43
[0, 6, 8, 4, 3, 2, 1, 7, 5, 9] 43
[0, 6, 8, 4, 3, 2, 1, 7, 5, 9] 43
divs
[1, 2, 3, 4, 8, 6, 0, 7, 5, 9] 53
[7, 0, 6, 8, 4, 3, 2, 1, 5, 9] 48
[7, 0, 6, 8, 4, 3, 2, 1, 5, 9] 48
[7, 0, 6, 8, 4, 3, 2, 1, 5, 9] 48
[7, 0, 6, 8, 4, 3, 2, 1, 5, 

([2, 7, 5, 9, 1, 4, 3, 8, 6, 0], 33, 281)

In [21]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [22]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [23]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            print('vns: ', solution, value, i)
            new_solution = shaking_func(graph, solution, k) #diversification    
            new_value = calculate_total_edge_length(graph, new_solution)
            print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)
            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i
                value = new_value
                solution = deepcopy(new_solution)
    return solution, value, best_i

In [24]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

vns:  [9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
post shaking:  [9, 1, 2, 0, 8, 4, 7, 5, 3, 6] 65 0
[9, 1, 2, 0, 8, 4, 7, 5, 3, 6] 63 0
[9, 1, 2, 0, 8, 4, 7, 5, 3, 6] 63 1
[9, 1, 2, 0, 8, 4, 7, 5, 3, 6] 63 2
[9, 1, 2, 0, 8, 4, 7, 5, 3, 6] 63 3
[9, 1, 2, 0, 5, 7, 4, 8, 3, 6] 57 4
[9, 1, 2, 0, 5, 7, 4, 8, 3, 6] 57 5
[9, 1, 2, 5, 0, 7, 4, 8, 3, 6] 55 6
[9, 1, 2, 5, 0, 7, 4, 8, 3, 6] 55 7
[9, 1, 2, 5, 0, 7, 4, 8, 3, 6] 55 8
[9, 1, 5, 2, 0, 7, 4, 8, 3, 6] 53 9
vns:  [9, 2, 5, 1, 0, 7, 4, 8, 3, 6] 49 0
post shaking:  [9, 6, 5, 1, 4, 7, 0, 8, 3, 2] 53 0
[9, 6, 5, 1, 4, 7, 0, 8, 3, 2] 63 0
[9, 6, 4, 1, 5, 7, 0, 8, 3, 2] 57 1
[7, 5, 1, 4, 6, 9, 0, 8, 3, 2] 53 2
[7, 5, 1, 4, 6, 9, 0, 8, 3, 2] 53 3
[7, 5, 1, 4, 6, 9, 0, 8, 3, 2] 53 4
[7, 5, 1, 4, 6, 9, 0, 8, 3, 2] 53 5
[7, 0, 9, 6, 4, 1, 5, 8, 3, 2] 47 6
[7, 0, 9, 6, 4, 1, 5, 8, 3, 2] 47 7
[7, 0, 9, 6, 1, 4, 5, 8, 3, 2] 46 8
[7, 0, 9, 6, 1, 4, 5, 8, 3, 2] 46 9
vns:  [7, 0, 9, 6, 1, 4, 5, 8, 3, 2] 46 1
post shaking:  [7, 0, 9, 6, 1, 4, 5, 8, 2, 3] 50 1
[

([2, 7, 5, 9, 1, 3, 4, 6, 0, 8], 33, 29)

In [25]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

vns:  [9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
post shaking:  [9, 1, 4, 0, 8, 2, 7, 6, 3, 5] 61 0
[9, 1, 4, 0, 8, 2, 7, 6, 3, 5] 63 0
[6, 7, 2, 8, 0, 4, 1, 9, 3, 5] 53 1
[6, 7, 2, 8, 0, 4, 1, 9, 3, 5] 53 2
[6, 7, 2, 5, 3, 9, 1, 4, 0, 8] 51 3
[6, 7, 2, 5, 3, 9, 1, 4, 0, 8] 51 4
[6, 7, 2, 5, 3, 9, 1, 4, 0, 8] 51 5
[6, 7, 2, 5, 3, 9, 1, 4, 0, 8] 51 6
[6, 7, 2, 5, 3, 9, 1, 4, 0, 8] 51 7
[6, 2, 7, 5, 3, 9, 1, 4, 0, 8] 49 8
[6, 2, 7, 5, 3, 9, 1, 4, 0, 8] 49 9
vns:  [6, 2, 7, 5, 3, 9, 1, 4, 0, 8] 49 0
post shaking:  [6, 2, 7, 4, 1, 9, 3, 5, 0, 8] 59 0
[6, 2, 7, 4, 1, 9, 3, 5, 0, 8] 63 0
[6, 2, 7, 4, 1, 3, 9, 5, 0, 8] 58 1
[6, 2, 7, 4, 9, 3, 1, 5, 0, 8] 52 2
[6, 2, 7, 3, 9, 4, 1, 5, 0, 8] 46 3
[6, 2, 7, 3, 9, 4, 1, 5, 0, 8] 46 4
[6, 2, 7, 3, 9, 4, 1, 5, 0, 8] 46 5
[3, 7, 2, 6, 9, 4, 1, 5, 0, 8] 44 6
[3, 7, 2, 6, 9, 4, 1, 5, 0, 8] 44 7
[3, 7, 2, 6, 9, 4, 1, 5, 0, 8] 44 8
[3, 7, 2, 6, 9, 4, 1, 5, 0, 8] 44 9
vns:  [3, 7, 2, 6, 9, 4, 1, 5, 0, 8] 44 0
post shaking:  [3, 7, 2, 6, 9, 8, 0, 5, 1, 4] 50 0
[

([2, 7, 5, 9, 1, 4, 3, 6, 8, 0], 33, 7)

In [26]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

vns:  [9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
post shaking:  [9, 1, 4, 0, 3, 2, 7, 5, 8, 6] 59 0
[9, 1, 4, 0, 3, 2, 7, 5, 8, 6] 63 0
[9, 0, 4, 1, 3, 2, 7, 5, 8, 6] 59 1
[0, 9, 4, 1, 3, 2, 7, 5, 8, 6] 56 2
[0, 9, 4, 1, 3, 6, 7, 5, 8, 2] 46 3
[0, 9, 4, 1, 3, 6, 7, 5, 8, 2] 46 4
[0, 9, 4, 1, 3, 6, 7, 5, 8, 2] 46 5
[0, 9, 4, 1, 3, 6, 7, 5, 8, 2] 46 6
[0, 9, 4, 1, 3, 6, 7, 5, 8, 2] 46 7
[0, 1, 4, 9, 3, 6, 7, 5, 8, 2] 42 8
[0, 1, 4, 9, 3, 6, 7, 5, 8, 2] 42 9
vns:  [0, 1, 4, 9, 3, 6, 7, 5, 8, 2] 42 0
post shaking:  [0, 1, 2, 7, 3, 6, 9, 5, 8, 4] 47 0
[0, 1, 2, 7, 3, 6, 9, 5, 8, 4] 63 0
[0, 1, 2, 7, 3, 8, 9, 5, 6, 4] 57 1
[0, 1, 2, 7, 3, 8, 9, 5, 6, 4] 57 2
[0, 1, 2, 7, 3, 5, 9, 8, 6, 4] 51 3
[0, 1, 2, 7, 3, 5, 9, 6, 8, 4] 47 4
[0, 1, 2, 7, 3, 5, 9, 6, 8, 4] 47 5
[0, 1, 2, 7, 3, 5, 9, 6, 8, 4] 47 6
[0, 1, 2, 7, 3, 5, 9, 6, 8, 4] 47 7
[0, 1, 2, 7, 3, 5, 9, 6, 8, 4] 47 8
[0, 1, 2, 7, 3, 5, 9, 6, 8, 4] 47 9
vns:  [0, 1, 4, 9, 3, 6, 7, 5, 8, 2] 42 1
post shaking:  [5, 1, 4, 9, 3, 6, 7, 0, 8, 2] 38 1
[

([2, 8, 0, 6, 3, 4, 1, 9, 7, 5], 33, 44)

In [27]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

vns:  [9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
post shaking:  [9, 1, 4, 0, 8, 2, 7, 6, 3, 5] 61 0
[9, 1, 4, 0, 8, 2, 7, 6, 3, 5] 63 0
[9, 1, 4, 0, 8, 2, 7, 6, 3, 5] 63 1
[9, 1, 4, 0, 8, 2, 7, 3, 6, 5] 61 2
[9, 1, 4, 0, 8, 2, 7, 3, 6, 5] 61 3
[9, 1, 4, 3, 8, 2, 7, 0, 6, 5] 59 4
[9, 1, 4, 3, 8, 2, 7, 0, 6, 5] 59 5
[9, 1, 4, 5, 8, 2, 7, 0, 6, 3] 55 6
[9, 1, 4, 5, 8, 2, 7, 0, 6, 3] 55 7
[9, 1, 4, 5, 8, 2, 7, 0, 6, 3] 55 8
[9, 1, 4, 5, 8, 2, 7, 0, 6, 3] 55 9
vns:  [9, 1, 4, 5, 8, 2, 7, 0, 6, 3] 55 0
post shaking:  [9, 1, 7, 2, 8, 5, 4, 0, 6, 3] 63 0
[9, 1, 7, 2, 8, 5, 4, 0, 6, 3] 63 0
[9, 1, 7, 2, 8, 5, 4, 0, 6, 3] 63 1
[9, 1, 7, 2, 8, 5, 4, 0, 6, 3] 63 2
[3, 1, 7, 2, 8, 5, 4, 0, 6, 9] 62 3
[3, 1, 7, 2, 0, 5, 4, 8, 6, 9] 58 4
[3, 1, 7, 2, 0, 5, 4, 8, 6, 9] 58 5
[3, 1, 7, 2, 0, 5, 4, 8, 6, 9] 58 6
[3, 7, 1, 2, 0, 5, 4, 8, 6, 9] 55 7
[3, 7, 1, 2, 0, 5, 4, 8, 6, 9] 55 8
[3, 7, 1, 9, 0, 5, 4, 8, 6, 2] 45 9
vns:  [3, 7, 1, 9, 0, 5, 4, 8, 6, 2] 45 0
post shaking:  [3, 7, 1, 9, 0, 2, 6, 8, 4, 5] 53 0
[

([2, 5, 7, 9, 1, 3, 4, 8, 6, 0], 33, 19)

In [28]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

vns:  [9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
post shaking:  [9, 1, 4, 0, 5, 2, 7, 8, 3, 6] 57 0
[9, 1, 4, 0, 5, 2, 7, 8, 3, 6] 63 0
[9, 1, 4, 3, 5, 2, 7, 8, 0, 6] 53 1
[9, 1, 4, 3, 5, 2, 7, 6, 0, 8] 51 2
[9, 1, 4, 3, 5, 2, 7, 6, 0, 8] 51 3
[9, 1, 4, 3, 5, 2, 7, 6, 0, 8] 51 4
[7, 1, 4, 3, 5, 2, 9, 6, 0, 8] 49 5
[7, 1, 4, 3, 5, 2, 9, 6, 0, 8] 49 6
[5, 1, 4, 3, 7, 2, 9, 6, 0, 8] 45 7
[5, 1, 4, 3, 7, 2, 9, 6, 0, 8] 45 8
[5, 1, 4, 3, 7, 2, 9, 6, 0, 8] 45 9
vns:  [5, 1, 4, 3, 7, 2, 9, 6, 0, 8] 45 0
post shaking:  [5, 1, 4, 3, 6, 2, 9, 7, 8, 0] 51 0
[5, 1, 4, 3, 6, 2, 9, 7, 8, 0] 63 0
[5, 7, 4, 3, 6, 2, 9, 1, 8, 0] 55 1
[5, 7, 1, 3, 6, 2, 9, 4, 8, 0] 50 2
[5, 7, 1, 3, 6, 2, 9, 4, 8, 0] 50 3
[5, 7, 1, 3, 6, 2, 9, 4, 8, 0] 50 4
[5, 7, 3, 1, 6, 2, 9, 4, 8, 0] 49 5
[5, 7, 3, 1, 6, 2, 9, 4, 8, 0] 49 6
[5, 7, 3, 1, 6, 0, 9, 4, 8, 2] 41 7
[5, 7, 3, 1, 6, 0, 9, 4, 8, 2] 41 8
[5, 7, 3, 9, 6, 0, 1, 4, 8, 2] 39 9
vns:  [5, 7, 3, 9, 6, 0, 1, 4, 8, 2] 39 1
post shaking:  [5, 7, 3, 9, 1, 0, 6, 4, 8, 2] 37 1
[

([0, 6, 8, 4, 3, 1, 9, 5, 7, 2], 33, 48)

In [29]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

vns:  [9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
post shaking:  [9, 8, 0, 4, 1, 2, 7, 5, 3, 6] 64 0
[9, 8, 0, 4, 1, 2, 7, 5, 3, 6] 63 0
[9, 8, 0, 4, 1, 2, 7, 5, 3, 6] 63 1
[9, 8, 0, 4, 1, 2, 7, 5, 3, 6] 63 2
[9, 8, 6, 4, 1, 2, 7, 5, 3, 0] 58 3
[9, 8, 6, 5, 1, 3, 0, 2, 4, 7] 56 4
[9, 8, 6, 5, 1, 3, 4, 2, 0, 7] 52 5
[9, 8, 6, 1, 5, 3, 4, 7, 0, 2] 51 6
[9, 8, 6, 1, 5, 3, 4, 7, 0, 2] 51 7
[9, 8, 6, 1, 5, 3, 4, 7, 0, 2] 51 8
[9, 8, 6, 1, 5, 3, 4, 7, 0, 2] 51 9
vns:  [9, 0, 6, 1, 5, 3, 4, 7, 8, 2] 45 0
post shaking:  [9, 0, 6, 1, 5, 3, 2, 8, 7, 4] 55 0
[9, 0, 6, 1, 5, 3, 2, 8, 7, 4] 63 0
[9, 0, 6, 1, 2, 3, 5, 8, 7, 4] 59 1
[9, 0, 6, 1, 2, 8, 5, 4, 3, 7] 55 2
[9, 0, 6, 1, 2, 8, 5, 4, 3, 7] 55 3
[5, 0, 6, 1, 2, 8, 9, 4, 3, 7] 45 4
[5, 0, 1, 6, 2, 8, 9, 4, 3, 7] 44 5
[5, 0, 1, 6, 2, 8, 9, 4, 3, 7] 44 6
[5, 0, 1, 6, 2, 8, 9, 4, 3, 7] 44 7
[5, 0, 1, 6, 2, 8, 9, 4, 3, 7] 44 8
[5, 0, 1, 6, 2, 8, 9, 4, 3, 7] 44 9
vns:  [5, 0, 1, 6, 2, 8, 9, 4, 3, 7] 44 0
post shaking:  [5, 9, 8, 2, 6, 1, 0, 4, 3, 7] 48 0
[

([8, 0, 6, 4, 3, 1, 9, 5, 7, 2], 33, 46)

In [30]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

vns:  [9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
post shaking:  [3, 1, 4, 0, 8, 2, 7, 5, 9, 6] 57 0
[3, 1, 4, 0, 8, 2, 7, 5, 9, 6] 63 0
[3, 1, 4, 0, 8, 2, 9, 5, 7, 6] 57 1
[5, 9, 2, 8, 0, 4, 1, 3, 7, 6] 53 2
[5, 9, 2, 8, 0, 4, 1, 3, 7, 6] 53 3
[5, 9, 8, 2, 0, 4, 1, 6, 3, 7] 51 4
[5, 9, 8, 6, 1, 4, 0, 2, 3, 7] 45 5
[5, 9, 8, 6, 1, 4, 0, 2, 3, 7] 45 6
[5, 9, 8, 6, 1, 4, 0, 2, 3, 7] 45 7
[5, 2, 0, 4, 1, 6, 9, 3, 7, 8] 44 8
[5, 2, 0, 4, 1, 6, 9, 3, 7, 8] 44 9
vns:  [8, 7, 3, 9, 6, 1, 4, 0, 5, 2] 42 0
post shaking:  [8, 7, 4, 9, 6, 2, 3, 0, 5, 1] 48 0
[8, 7, 4, 9, 6, 2, 3, 0, 5, 1] 63 0
[8, 9, 4, 7, 6, 2, 3, 0, 5, 1] 52 1
[8, 9, 4, 7, 6, 2, 3, 0, 5, 1] 52 2
[8, 9, 4, 7, 6, 2, 3, 0, 5, 1] 52 3
[8, 9, 4, 7, 6, 1, 5, 0, 3, 2] 48 4
[8, 7, 4, 9, 6, 1, 5, 0, 3, 2] 44 5
[8, 7, 4, 9, 6, 1, 5, 0, 3, 2] 44 6
[8, 7, 4, 9, 6, 1, 5, 0, 3, 2] 44 7
[8, 7, 4, 9, 6, 3, 0, 5, 1, 2] 41 8
[8, 7, 4, 9, 6, 3, 0, 5, 1, 2] 41 9
vns:  [8, 7, 4, 9, 6, 3, 0, 5, 1, 2] 41 1
post shaking:  [8, 7, 2, 9, 6, 3, 0, 5, 1, 4] 49 1
[

([0, 8, 6, 3, 4, 1, 9, 5, 7, 2], 33, 1)

In [31]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

vns:  [9, 1, 4, 0, 8, 2, 7, 5, 3, 6] 63 0
post shaking:  [9, 8, 0, 4, 1, 2, 7, 5, 3, 6] 64 0
[9, 8, 0, 4, 1, 2, 7, 5, 3, 6] 63 0
[9, 6, 3, 5, 7, 2, 1, 4, 0, 8] 56 1
[9, 6, 3, 0, 4, 1, 5, 2, 7, 8] 55 2
[2, 5, 1, 4, 0, 3, 6, 9, 7, 8] 41 3
[2, 5, 1, 4, 0, 3, 6, 9, 7, 8] 41 4
[2, 5, 1, 4, 0, 3, 6, 9, 7, 8] 41 5
[2, 5, 1, 4, 0, 3, 6, 9, 7, 8] 41 6
[2, 5, 1, 4, 0, 3, 6, 9, 7, 8] 41 7
[2, 5, 1, 4, 0, 3, 6, 9, 7, 8] 41 8
[2, 5, 1, 4, 0, 3, 6, 9, 7, 8] 41 9
vns:  [2, 5, 1, 4, 0, 3, 6, 9, 7, 8] 41 0
post shaking:  [2, 3, 0, 4, 1, 5, 6, 9, 7, 8] 45 0
[2, 3, 0, 4, 1, 5, 6, 9, 7, 8] 63 0
[4, 0, 3, 2, 1, 5, 6, 9, 7, 8] 53 1
[4, 0, 3, 2, 1, 5, 6, 9, 7, 8] 53 2
[0, 4, 3, 2, 1, 5, 6, 9, 7, 8] 51 3
[0, 4, 3, 2, 1, 5, 6, 9, 7, 8] 51 4
[0, 4, 3, 2, 1, 5, 6, 9, 7, 8] 51 5
[2, 3, 4, 0, 1, 5, 6, 9, 7, 8] 45 6
[2, 1, 0, 4, 3, 5, 6, 9, 7, 8] 44 7
[2, 1, 0, 4, 3, 5, 6, 9, 7, 8] 44 8
[8, 7, 9, 6, 5, 3, 4, 1, 0, 2] 41 9
vns:  [2, 5, 1, 4, 0, 3, 6, 9, 7, 8] 41 0
post shaking:  [2, 5, 1, 7, 9, 6, 3, 0, 4, 8] 40 0
[

([8, 6, 0, 3, 4, 1, 9, 5, 7, 2], 33, 12)

In [32]:
#genetic algorithms

In [33]:
class Individual:
    def __init__(self, num_nodes, graph):
        self.num_nodes = num_nodes
        self.graph = graph
        self.code = self.generate_solution(self.graph)
        self.fitness = self.calc_fitness(self.code)
    
    def generate_solution(self, num_nodes):
        nodes = list(range(num_nodes))
        random.shuffle(nodes)
        return nodes

    def calc_fitness(self, solution):
        total_length = 0
        for i in range(len(graph)):
            for j in range(i + 1, len(graph)):
                if graph[i][j] == 1:
                    position_i = solution.index(i)
                    position_j = solution.index(j)
                    total_length += abs(position_i - position_j)
        return total_length

In [34]:
def tournament_selection(population, tour_size):
    tournament = random.sample(population, tour_size)
    selected = max(tournament, key=lambda x: x.fitness)
    return selected

In [42]:
#beacause we are dealing with permutations, we have to be careful here and used sepcialized crossover algorithms
#ordered crossover method
#PROVERI OVO
def ordered_crossover(parent1, parent2, cx_point1 = -1, cx_point2 = -1):
    n = len(parent1)

    # Choose two random crossover points
    if cx_point1 == -1 and cx_point2 == -1:
        cx_point1, cx_point2 = sorted(random.sample(range(n), 2))
        print(cx_point1, cx_point2)

    # Copy the segment between the crossover points from parent1 to child1
    child1_segment = parent1[cx_point1:cx_point2 + 1]

    # Fill the remaining positions in child1 with elements from parent2
    child1 = [-1] * n
    child1[cx_point1:cx_point2 + 1] = child1_segment
    remaining_positions = [i for i in parent2 if i not in child1_segment]
    j = 0
    for i in range(n):
        if child1[i] == -1:
            child1[i] = remaining_positions[j]
            j += 1

    return child1, cx_point1, cx_point2


In [44]:
child1, i1, i2 = ordered_crossover([5, 4, 1, 3, 6, 8, 9, 0, 2, 7], [8, 0, 7, 6, 4, 5, 3, 1, 9, 2])
child2, _, _ = ordered_crossover([8, 0, 7, 6, 4, 5, 3, 1, 9, 2],[5, 4, 1, 3, 6, 8, 9, 0, 2, 7], i1, i2)
print(child1, child2)

1 5
[0, 4, 1, 3, 6, 8, 7, 5, 9, 2] [1, 0, 7, 6, 4, 5, 3, 8, 9, 2]


In [45]:
def mutation_swap():
    pass

In [46]:
def mutation_inverse():
    pass